In [16]:
import pandas as pd
import numpy as np
from json import loads, dumps
from IPython.display import display
from ortools.constraint_solver import routing_enums_pb2, pywrapcp
from scipy.spatial import distance_matrix
import subprocess

In [17]:
inf = int(1e10)
max_time = (24 - 6) * 60
start = 6 * 60

In [18]:
#DATA_IN_PATH = "phystech-master/data/contest_input.json"
#DATA_OUT_PATH = "phystech-master/data/model_output.json"

#DATA_IN_PATH = "phystech-master/data/simple_input.json"
#DATA_OUT_PATH = "phystech-master/data/model_output.json"

DATA_IN_PATH = "phystech-master/kamil/cls_input.json"
DATA_OUT_PATH = "phystech-master/kamil/cls_output.json"

In [19]:
with open(DATA_IN_PATH, 'r') as file:
    data_in_json = file.read()

In [20]:
data_in_python = loads(data_in_json)
columns = list(data_in_python.keys())
columns

['orders', 'depots', 'couriers']

In [21]:
dfs = {}
for key, value in data_in_python.items():
    dfs[key] = pd.DataFrame(value)

couriers = dfs["couriers"]
depots = dfs["depots"]
orders = dfs["orders"]

In [22]:
places = []
places_simple = []
pickups_deliveries = []
time_windows = []

'''
for _, depot in depots.iterrows():
    place = {"point_id": depot["point_id"], "x": depot["location_x"], "y": depot["location_y"], "type": "depot"}
    places_simple.append([place["x"], place["y"]])
    time_windows.append([0, max_time])
    places.append(place)
'''

for i, order in orders.iterrows():    
    place = {"point_id": order["pickup_point_id"], "x": order["pickup_location_x"], "y": order["pickup_location_y"], "from": order["pickup_from"], "to": order["pickup_to"], "type": "pickup"}
    place2 = {"point_id": order["dropoff_point_id"], "x": order["dropoff_location_x"], "y": order["dropoff_location_y"], "from": order["dropoff_from"], "to": order["dropoff_to"], "type": "dropoff"}
    if place["from"] >= place["to"] or place2["from"] >= place2["to"]:
        orders.drop(i)
        continue
    
    pickups_deliveries.append([len(places_simple), len(places_simple) + 1])
    
    places_simple.append([place["x"], place["y"]])
    time_windows.append([place["from"] - start, place["to"] - start])
    places.append(place)
    
    places_simple.append([place2["x"], place2["y"]])
    time_windows.append([place2["from"] - start, place2["to"] - start])
    places.append(place2)

route_start = len(places_simple)
for _, courier in couriers.iterrows():
    places_simple.append([courier["location_x"], courier["location_y"]])
    time_windows.append([0, max_time])

places = pd.DataFrame(places)
distances = distance_matrix(x=places_simple, y=places_simple, p=1) + 10
distances = np.append(distances, np.zeros(distances.shape[0]).reshape(-1, 1), axis=1)
distances = np.append(distances, np.zeros(distances.shape[1]).reshape(1, -1), axis=0)
time_windows.append([0, max_time])

In [23]:
orders.drop("pickup_location_x", axis=1, inplace=True)
orders.drop("pickup_location_y", axis=1, inplace=True)
orders.drop("pickup_from", axis=1, inplace=True)
orders.drop("pickup_to", axis=1, inplace=True)

orders.drop("dropoff_location_x", axis=1, inplace=True)
orders.drop("dropoff_location_y", axis=1, inplace=True)
orders.drop("dropoff_from", axis=1, inplace=True)
orders.drop("dropoff_to", axis=1, inplace=True)

In [24]:
display(couriers)
display(places)
display(orders)

display(distances)
display(time_windows)
display(pickups_deliveries)

,courier_id,location_x,location_y
0,1,111,5
1,2,260,112
2,3,207,66
3,4,288,55
4,5,253,234
...,...,...,...
295,296,266,56
296,297,214,124
297,298,229,143
298,299,106,150


,point_id,x,y,from,to,type
0,40021,201,143,570,600,pickup
1,60021,236,126,660,810,dropoff
2,40038,201,143,570,600,pickup
3,60038,236,126,660,810,dropoff
4,40057,188,136,540,840,pickup
...,...,...,...,...,...,...
459,67155,281,122,959,1110,dropoff
460,47163,118,141,360,1020,pickup
461,67163,207,117,1020,1080,dropoff
462,47300,196,112,330,1140,pickup


,order_id,pickup_point_id,dropoff_point_id,payment,done,pickup_window,dropoff_window,vec_x,vec_y,pickup_mid,dropoff_mid,delivery_time
0,10021,40021,60021,342,0,30,150,35,-17,1200,1620,62
1,10038,40038,60038,342,0,30,150,35,-17,1200,1620,62
2,10057,40057,60057,188,0,300,300,43,-16,1680,1680,69
3,10059,40059,60059,403,0,60,60,36,-15,1020,1080,61
4,10067,40067,60067,403,0,60,60,36,-15,1020,1080,61
...,...,...,...,...,...,...,...,...,...,...,...,...
227,17072,47072,67072,475,0,630,122,69,-30,2040,2100,109
228,17108,47108,67108,412,0,660,144,75,-1,2040,2160,86
229,17155,47155,67155,220,0,780,151,53,-3,2160,2220,66
230,17163,47163,67163,329,0,660,60,89,-24,2040,2160,123


array([[ 10.,  62.,  10., ..., 112.,  41.,   0.],
       [ 62.,  10.,  62., ..., 164.,  31.,   0.],
       [ 10.,  62.,  10., ..., 112.,  41.,   0.],
       ...,
       [112., 164., 112., ...,  10., 143.,   0.],
       [ 41.,  31.,  41., ..., 143.,  10.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])

[[210, 240],
 [300, 450],
 [210, 240],
 [300, 450],
 [180, 480],
 [180, 480],
 [90, 150],
 [120, 180],
 [90, 150],
 [120, 180],
 [60, 90],
 [90, 240],
 [60, 300],
 [120, 360],
 [30, 90],
 [30, 360],
 [180, 240],
 [300, 360],
 [180, 540],
 [180, 540],
 [0, 240],
 [0, 240],
 [90, 120],
 [210, 270],
 [60, 120],
 [120, 180],
 [30, 60],
 [60, 120],
 [0, 120],
 [180, 210],
 [240, 480],
 [240, 480],
 [60, 300],
 [120, 300],
 [60, 180],
 [180, 360],
 [0, 180],
 [60, 240],
 [0, 60],
 [60, 420],
 [0, 300],
 [60, 480],
 [-60, 0],
 [-60, 150],
 [60, 300],
 [120, 360],
 [270, 300],
 [300, 420],
 [0, 240],
 [60, 300],
 [30, 60],
 [120, 150],
 [0, 30],
 [60, 90],
 [30, 120],
 [60, 300],
 [0, 60],
 [60, 120],
 [150, 210],
 [240, 300],
 [-59, 0],
 [-59, 30],
 [25, 210],
 [25, 360],
 [90, 120],
 [150, 180],
 [35, 180],
 [90, 240],
 [41, 120],
 [41, 150],
 [43, 120],
 [43, 150],
 [43, 120],
 [43, 180],
 [90, 120],
 [90, 222],
 [47, 120],
 [47, 180],
 [55, 120],
 [55, 150],
 [70, 300],
 [120, 300],
 [62, 

[[0, 1],
 [2, 3],
 [4, 5],
 [6, 7],
 [8, 9],
 [10, 11],
 [12, 13],
 [14, 15],
 [16, 17],
 [18, 19],
 [20, 21],
 [22, 23],
 [24, 25],
 [26, 27],
 [28, 29],
 [30, 31],
 [32, 33],
 [34, 35],
 [36, 37],
 [38, 39],
 [40, 41],
 [42, 43],
 [44, 45],
 [46, 47],
 [48, 49],
 [50, 51],
 [52, 53],
 [54, 55],
 [56, 57],
 [58, 59],
 [60, 61],
 [62, 63],
 [64, 65],
 [66, 67],
 [68, 69],
 [70, 71],
 [72, 73],
 [74, 75],
 [76, 77],
 [78, 79],
 [80, 81],
 [82, 83],
 [84, 85],
 [86, 87],
 [88, 89],
 [90, 91],
 [92, 93],
 [94, 95],
 [96, 97],
 [98, 99],
 [100, 101],
 [102, 103],
 [104, 105],
 [106, 107],
 [108, 109],
 [110, 111],
 [112, 113],
 [114, 115],
 [116, 117],
 [118, 119],
 [120, 121],
 [122, 123],
 [124, 125],
 [126, 127],
 [128, 129],
 [130, 131],
 [132, 133],
 [134, 135],
 [136, 137],
 [138, 139],
 [140, 141],
 [142, 143],
 [144, 145],
 [146, 147],
 [148, 149],
 [150, 151],
 [152, 153],
 [154, 155],
 [156, 157],
 [158, 159],
 [160, 161],
 [162, 163],
 [164, 165],
 [166, 167],
 [168, 169],
 [170

In [25]:
print("couriers:", couriers.shape)
print("places:", places.shape)
print("orders:", orders.shape)

couriers: (300, 3)
places: (464, 6)
orders: (232, 12)


In [26]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    max_route_distance = 0
    json = []
    
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        
        plan_output += 'START ->';
        while not routing.IsEnd(index):
            if manager.IndexToNode(index) < places.index.stop:
                place = places.iloc[manager.IndexToNode(index)]
                plan_output += ' {} -> '.format(place["point_id"])
                
                order_id = orders[((orders["pickup_point_id"] == place["point_id"]) | (orders["dropoff_point_id"] == place["point_id"]))]["order_id"]
                if len(order_id) != 0:
                    order_id = order_id.head(1)
                else:
                    order_id = -1
                
                current_json = {
                    "courier_id": int(couriers.iloc[vehicle_id]["courier_id"]),
                    "action": place["type"], # check depot
                    "order_id": int(order_id),
                    "point_id": int(place["point_id"]),
                }
                json.append(current_json)
                
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        
        if manager.IndexToNode(index) < places.index.stop:
            plan_output += '{}\n'.format(manager.IndexToNode(index))
        else:
            plan_output += 'END\n'
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))
    return json

In [27]:
data = {}
data["time_matrix"] = distances
data["time_windows"] = time_windows
data["num_vehicles"] = len(couriers)
data["starts"] = list(range(route_start, len(distances) - 1))
data["ends"] = [len(distances) - 1] * len(couriers)
data["pickups_deliveries"] = pickups_deliveries

In [28]:
%%time
manager = pywrapcp.RoutingIndexManager(len(data["time_matrix"]), data["num_vehicles"], data["starts"], data["ends"])
routing = pywrapcp.RoutingModel(manager)

def time_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data["time_matrix"][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(time_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

dimension_name = 'Time'
routing.AddDimension(
    transit_callback_index,
    max_time,  # inf slack
    max_time,  # vehicle maximum travel distance
    False,  # start cumul to zero
    dimension_name)

time_dimension = routing.GetDimensionOrDie(dimension_name)

for location_idx in range(route_start):    
    time_window = data["time_windows"][location_idx]
    index = manager.NodeToIndex(location_idx)
    time_dimension.CumulVar(index).SetRange(int(time_window[0]), int(time_window[1]))

for vehicle_id in range(data['num_vehicles']):
    index = routing.Start(vehicle_id)
    time_dimension.CumulVar(index).SetRange(int(data['time_windows'][-1][0]),
                                            int(data['time_windows'][-1][1]))

for i in range(data['num_vehicles']):
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.Start(i)))
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.End(i)))

for i, place in places.iterrows():
    order_payment = orders[((orders["pickup_point_id"] == place["point_id"]) | (orders["dropoff_point_id"] == place["point_id"]))]["payment"]
    if len(order_payment) != 0:
        order_payment = order_payment.head(1)
    else:
        order_payment = 0
    routing.AddDisjunction([manager.NodeToIndex(i)], int(order_payment))

for request in data["pickups_deliveries"]:
    pickup_index = manager.NodeToIndex(request[0])
    delivery_index = manager.NodeToIndex(request[1])
    routing.AddPickupAndDelivery(pickup_index, delivery_index)
    routing.solver().Add(
        routing.VehicleVar(pickup_index) == routing.VehicleVar(
            delivery_index))
    routing.solver().Add(
        time_dimension.CumulVar(pickup_index) <=
        time_dimension.CumulVar(delivery_index))

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.time_limit.seconds = 60 * 60
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

solution = routing.SolveWithParameters(search_parameters)

CPU times: user 1min 33s, sys: 12.2 ms, total: 1min 33s
Wall time: 1min 33s


In [29]:
if solution:
    print(routing.status())
    
    json = print_solution(data, manager, routing, solution)
    with open(DATA_OUT_PATH, "w") as out:
        out.write(dumps(json))
    print("Done.")

1
Route for vehicle 0:
START ->END
Distance of the route: 0m

Route for vehicle 1:
START ->END
Distance of the route: 0m

Route for vehicle 2:
START ->END
Distance of the route: 0m

Route for vehicle 3:
START ->END
Distance of the route: 0m

Route for vehicle 4:
START ->END
Distance of the route: 0m

Route for vehicle 5:
START ->END
Distance of the route: 0m

Route for vehicle 6:
START ->END
Distance of the route: 0m

Route for vehicle 7:
START ->END
Distance of the route: 0m

Route for vehicle 8:
START -> 47067 ->  43798 ->  44256 ->  44201 ->  63798 ->  64201 ->  64256 ->  67067 -> END
Distance of the route: 237m

Route for vehicle 9:
START ->END
Distance of the route: 0m

Route for vehicle 10:
START -> 40260 ->  45290 ->  60260 ->  65290 -> END
Distance of the route: 161m

Route for vehicle 11:
START ->END
Distance of the route: 0m

Route for vehicle 12:
START -> 43574 ->  42679 ->  46859 ->  62679 ->  63574 ->  66859 -> END
Distance of the route: 181m

Route for vehicle 13:
START -


Route for vehicle 182:
START ->END
Distance of the route: 0m

Route for vehicle 183:
START -> 46944 ->  44484 ->  44664 ->  40143 ->  46482 ->  64484 ->  64664 ->  66482 ->  60143 ->  66944 -> END
Distance of the route: 242m

Route for vehicle 184:
START -> 42970 ->  43600 ->  62970 ->  63600 -> END
Distance of the route: 164m

Route for vehicle 185:
START ->END
Distance of the route: 0m

Route for vehicle 186:
START -> 43017 ->  44227 ->  46707 ->  63017 ->  64227 ->  66707 -> END
Distance of the route: 126m

Route for vehicle 187:
START ->END
Distance of the route: 0m

Route for vehicle 188:
START ->END
Distance of the route: 0m

Route for vehicle 189:
START ->END
Distance of the route: 0m

Route for vehicle 190:
START ->END
Distance of the route: 0m

Route for vehicle 191:
START ->END
Distance of the route: 0m

Route for vehicle 192:
START ->END
Distance of the route: 0m

Route for vehicle 193:
START -> 46750 ->  45286 ->  44292 ->  65286 ->  64292 ->  66750 -> END
Distance of the 

Done.


In [30]:
try:
    out = subprocess.check_output(["python3", "check.py", DATA_IN_PATH, DATA_OUT_PATH]).decode("ascii")
    out_profit = int(out[out.find("Profit: ") + 8:])
    
    print("Profit:", out_profit)

    if out_profit < 0:
        with open(DATA_OUT_PATH, "w") as out:
            out.write(dumps([]))
        print("Rewritten.")
except Exception as ex:
    print("Test failed:\n", ex)

Profit: 10113


# Profits

### Example
Example: 2438  
No clustering / depots: **2498**  

### Simple
Example: 1380  
No clustering / depots: **1596** 

### Hard
No clustering / depots: **1952**